In [1]:
import json, operator, os, re, requests, string, pandas as pd
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

In [2]:
with open('../data/processed/hamilton_data.json', 'r') as f:
    haml = json.load(f)

In [3]:
for key in haml[0]:
    print(key)

act#
lyrics
track#
track


In [4]:
for song in haml:
    song['act#'] = int(song['act#'])
    song['track#'] = int(song['track#'])
haml.sort(key=operator.itemgetter('act#','track#'))
#for song in haml:
#    print('Act:', song['act#'], 'Track#:', song['track#'], 'Track:', song['track'])

In [5]:
basestats = json_normalize(haml)[['act#', 'track#','track','lyrics']]

In [6]:
line_length, word_length = [], []
for song in haml:    
    line_length.append(len(song['lyrics']))
    words_in_song = 0
    for line in song['lyrics']:
        words_in_song += len(line['tokenized'])
    word_length.append(words_in_song)    
basestats['#lines'] = line_length
basestats['#words'] = word_length

In [7]:
r = requests.get('https://en.wikipedia.org/wiki/Hamilton_(album)')
soup = BeautifulSoup(r.content, 'html.parser')
tbl1, tbl2 = soup.find_all('table')[2], soup.find_all('table')[3]
rows1, rows2 = tbl1.find_all('tr'), tbl2.find_all('tr')

In [8]:
song_length = []
for row in rows1[:-1]:  # Exclude last line, cause it's just a summary
    if len(row.find_all('td'))==0:
        continue  # Exclude empty rows
    row_compl = row.find_all('td')  # Find all cells
    song_length.append(row_compl[3].text)  # Get row giving the length of song
for row in rows2[:-1]:  # Exclude last line, cause it's just a summary
    if len(row.find_all('td'))==0:
        continue  # Exclude empty rows
    row_compl = row.find_all('td')  # Find all cells
    song_length.append(row_compl[3].text)  # Get row giving the length of song
basestats['len_seconds'] = [int(m[0])*60+int(m[2:4]) for m in song_length]

In [9]:
all_speakers = []
speakers_song_ls = []
for song in haml:
    speakers_song = []
    for line in song['lyrics']:
        all_speakers.append(line['speakers'])
        speakers_song.append(line['speakers'])
    #speakers_song_ls.append(set(speakers_song))
    speakers_song_ls.append(set([item for sublist in speakers_song for item in sublist]))
    #print(set([item for sublist in speakers_song for item in sublist]))
all_speakers = set([item for sublist in all_speakers for item in sublist])
basestats['speakers'] = speakers_song_ls

In [43]:
num_speakers = []
for spkls in speakers_song_ls:
    num_speakers.append(len(spkls))
basestats['#speakers'] = num_speakers

In [44]:
basestats['#words/sec'] = basestats['#words']/basestats['len_seconds']

In [45]:
basestats.drop('lyrics', axis=1, inplace=False).to_csv('../data/processed/basestats.csv', sep=',')

In [46]:
basestats

,act#,track#,track,lyrics,#lines,#words,len_seconds,speakers,#speakers,#words/sec
0,1,1,Alexander Hamilton,"[{'speakers': ['BURR'], 'original': 'How does ...",93,615,236,"{JEFFERSON, WOMEN, PEGGY, BURR, MEN, FULL COMP...",20,2.605932
1,1,2,"Aaron Burr, Sir","[{'speakers': ['COMPANY'], 'original': '1776. ...",66,456,156,"{BURR, COMPANY, LAURENS, LAFAYETTE, HAMILTON, ...",6,2.923077
2,1,3,My Shot,"[{'speakers': ['HAMILTON'], 'original': 'I am ...",192,1198,333,"{BURR, COMPANY, LAURENS, LAFAYETTE, HAMILTON, ...",7,3.597598
3,1,4,The Story Of Tonight,"[{'speakers': ['HAMILTON'], 'original': 'I may...",32,200,91,"{LAURENS, LAFAYETTE, HAMILTON, MULLIGAN, FULL ...",6,2.197802
4,1,5,The Schuyler Sisters,"[{'speakers': ['BURR'], 'original': 'There�s n...",122,613,186,"{SCHUYLER SISTERS, PEGGY, WOMEN, BURR, MEN, FU...",13,3.295699
5,1,6,Farmer Refuted,"[{'speakers': ['SEABURY'], 'original': 'Hear y...",56,300,112,"{SEABURY, BURR, FULL COMPANY, COMPANY, HAMILTO...",7,2.678571
6,1,7,You'll Be Back,"[{'speakers': ['KING GEORGE'], 'original': 'Yo...",44,349,208,"{KING GEORGE, FULL ENSEMBLE}",2,1.677885
7,1,8,Right Hand Man,"[{'speakers': ['COMPANY'], 'original': 'Britis...",180,988,321,"{SCHUYLER SISTERS, ENSEMBLE 2, PEGGY, WOMEN, B...",15,3.077882
8,1,9,A Winter's Ball,"[{'speakers': ['BURR'], 'original': 'How does ...",26,137,69,"{HAMILTON, ALL MEN, BURR, LAURENS, FULL COMPANY}",5,1.985507
9,1,10,Helpless,"[{'speakers': ['HAMILTON', 'BURR', 'LAURENS', ...",150,803,249,"{WOMEN, BURR, ALL WOMEN, ELIZA, LAURENS, HAMIL...",7,3.224900


In [39]:
r = requests.get('https://en.wikipedia.org/w/index.php?title=Hamilton_(musical)&oldid=751273119')
soup = BeautifulSoupulSoup(r.content, 'html.parser')
cast = soup.find_all('table')[4]
castr = cast.find_all('tr')

In [47]:
main_cast = []
for row in castr:
    if len(row.find_all('th'))==0:
        continue  # Exclude empty rows
    row_compl = row.find_all('th')  # Find all cells
    cur_speaker_row = row_compl[0].text.split(' / ')
    for whole_name in cur_speaker_row:
        whole_name_list = whole_name.split(' ')
        if 'Character' in whole_name_list:
            continue
        # Women, children, and kings are referred to by first name
        elif 'Schuyler' in whole_name_list:
            main_cast.append(whole_name_list[0].upper())
        elif 'Reynolds' in whole_name_list:          
            main_cast.append(whole_name_list[0].upper())
        elif 'Philip' in whole_name_list:          
            main_cast.append(whole_name_list[0].upper())            
        elif 'King' in whole_name_list:
            main_cast.append('KING GEORGE'.upper())        
        else:
            main_cast.append(whole_name_list[-1].upper())

In [48]:
main_cast

['HAMILTON',
 'BURR',
 'ELIZA',
 'ANGELICA',
 'LAFAYETTE',
 'JEFFERSON',
 'WASHINGTON',
 'KING GEORGE',
 'LAURENS',
 'PHILIP',
 'PEGGY',
 'MARIA',
 'MULLIGAN',
 'MADISON']

In [49]:
all_corpora = dict()
for speaker in main_cast:
     all_corpora[speaker] = []
for song in haml:    
    for line in song['lyrics']:
        for speaker in line['speakers']:
            if speaker in main_cast:
                all_corpora[speaker].append(line['normalized'])

word_c = []
for speaker in main_cast:
    all_corpora[speaker] = [item for sublist in all_corpora[speaker] for item in sublist]
    print(speaker, ': ', len(all_corpora[speaker]))
    word_c.append(len(all_corpora[speaker]))
    
word_counts = pd.DataFrame(word_c,index=main_cast,columns=['#words'])
word_counts.to_csv('../data/processed/word_counts.csv', sep=',')

HAMILTON :  6154
BURR :  4040
ELIZA :  1808
ANGELICA :  1455
LAFAYETTE :  730
JEFFERSON :  1561
WASHINGTON :  1666
KING GEORGE :  588
LAURENS :  879
PHILIP :  572
PEGGY :  156
MARIA :  137
MULLIGAN :  584
MADISON :  739


In [24]:
def rem_stopw_punct(string_):
    # Remove common stopwords
    stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all",
                  "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst",
                  "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway",
                  "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes",
                  "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides",
                  "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co",
                  "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due",
                  "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough",
                  "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few",
                  "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty",
                  "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt",
                  "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers",
                  "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc",
                  "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly",
                  "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more",
                  "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither",
                  "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                  "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other",
                  "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
                  "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious",
                  "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                  "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such",
                  "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence",
                  "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv",
                  "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus",
                  "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under",
                  "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever",
                  "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
                  "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose",
                  "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself",
                  "yourselves", "the"]
    pattern1 = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
    string_ = pattern1.sub('', string_)

    # Remove punctuation, i.e. anything that's not a letter or space
    pattern2 = re.compile(r'[^A-Za-z ]')
    string_ = pattern2.sub(' ', string_).strip()

    # Also, reduce any sequence of 2+ white spaces to a single one, while also stripping trailing blanks
    pattern3 = re.compile(r'  +')
    string_ = pattern3.sub(' ', string_).strip()
    
    return string_.translate(string.punctuation)

In [25]:
def bool_sim(st1, st2, df):
    return (int(df[[st1]].multiply(df[st2], axis="index").sum()))/(int(df[[st1]].sum()) * int(df[[st2]].sum()))

In [26]:
for speaker in all_corpora:
    all_corpora[speaker] = rem_stopw_punct(str(' '.join(all_corpora[speaker])).lower())

In [27]:
# Get vocabulary
corp_flat = []
for speaker in all_corpora:
    corp_flat.append(all_corpora[speaker].split(' '))
corp_flat = [item for sublist in corp_flat for item in sublist]
voc = set(corp_flat)

# Create term-document matrix
tdm = pd.DataFrame(0,index=voc,columns=main_cast)
for speaker in all_corpora:
    tokens = all_corpora[speaker].split(' ')
    for word in tokens:
        tdm.at[word,speaker] = 1
tdm.to_csv('../data/processed/tdm.csv', sep=',')

In [28]:
bool_sim(main_cast[0], main_cast[2], tdm)

IndexError: list index out of range

In [29]:
# Do boolean, td-idf similarity
### STOPPED HERE
# Create document similarity matrix
dsm = pd.DataFrame(0,index=main_cast,columns=main_cast)
counter = 0
for cast1 in main_cast:
    for cast2_index in range(counter,len(main_cast)):
        if (main_cast == main_cast[cast2_index]):
            dsm.loc[main_cast, main_cast[cast2_index]] = 1
        else:
            dsm.loc[main_cast, main_cast[cast2_index]] = \
            dsm.loc[main_cast[cast2_index], main_cast] = \
            bool_sim(main_cast, main_cast[cast2_index], tdm)
    counter += 1
dsm.to_csv('../data/processed/dsm.csv', sep=' ')